# Audio Voice Separation Notebook

This notebook leverages [Demucs](https://github.com/facebookresearch/demucs) to separate vocals from an audio file.

### Use Cases
- **Voice Separation**: Create karaoke tracks or remix songs by isolating vocals.
- **Instrumental Extraction**: Remove vocals for backing tracks.
- **Audio Analysis**: Improve clarity for machine learning or podcast editing.

In [ ]:
# Install Demucs and required libraries
!pip install -q demucs
!pip install -q pydub

In [ ]:
import os
import sys
import subprocess
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")

from pydub import AudioSegment
from IPython.display import Audio, display

from google.colab import files

In [ ]:
def convert_to_wav(input_path, output_path):
    try:
        audio = AudioSegment.from_file(input_path)
        audio.export(output_path, format='wav')
        print(f"Successfully converted {input_path} to WAV format")
        return True
    except Exception as e:
        print(f"Error converting file: {str(e)}")
        return False


def separate_audio(input_file):
    try:
        print("Starting audio separation...")
        
        # Create output directory
        os.makedirs('separated', exist_ok=True)
        
        # Convert to wav if needed
        if not input_file.endswith('.wav'):
            wav_file = input_file.rsplit('.', 1)[0] + '.wav'
            if not convert_to_wav(input_file, wav_file):
                return False
            input_file = wav_file
        
        # Run Demucs separation
        !demucs --two-stems=vocals "$input_file"
        
        return True
    except Exception as e:
        print(f"Error in separation: {str(e)}")
        return False

In [ ]:
# Main execution
print("Please upload your MP3 file:")
uploaded = files.upload()

if not uploaded:
    print("No file was uploaded.")
    sys.exit()

input_file = list(uploaded.keys())[0]
print(f"Processing {input_file}...")

if separate_audio(input_file):
    try:
        # Get base name without extension
        base_name = os.path.splitext(input_file)[0]
        
        # Define Demucs output paths
        separated_dir = Path('separated/htdemucs') / base_name
        vocals_path = str(separated_dir / 'vocals.wav')
        no_vocals_path = str(separated_dir / 'no_vocals.wav')
        
        # Define MP3 output paths
        vocals_mp3 = vocals_path.replace('.wav', '.mp3')
        no_vocals_mp3 = no_vocals_path.replace('.wav', '.mp3')
        
        if os.path.exists(vocals_path) and os.path.exists(no_vocals_path):
            # Convert separated WAV files to MP3
            vocals = AudioSegment.from_wav(vocals_path)
            no_vocals = AudioSegment.from_wav(no_vocals_path)
            
            vocals.export(vocals_mp3, format='mp3')
            no_vocals.export(no_vocals_mp3, format='mp3')
            
            print("\nSeparation completed successfully!")
            
            print("\nVocals:")
            display(Audio(vocals_path))
            
            print("\nWithout Vocals:")
            display(Audio(no_vocals_path))
            
            print("\nDownloading separated files...")
            files.download(vocals_mp3)
            files.download(no_vocals_mp3)
            
            print("\nCleaning up temporary files...")
            if os.path.exists(input_file.replace('.mp3', '.wav')):
                os.remove(input_file.replace('.mp3', '.wav'))
        else:
            print("Error: Separated files not found. Separation may have failed.")
            
    except Exception as e:
        print(f"Error in final processing: {str(e)}")
else:
    print("Audio separation failed. Please try again.")